This notebook will:
1. Query Gaia DR3 for astrometry (parallax, proper motions, radial velocity)
2. Crossmatch with the Andrae et al. 2023 XP metallicity table

In [12]:
import numpy as np
import re
from astropy.table import Table
import matplotlib.pyplot as plt
import pandas as pd
from astroquery.gaia import Gaia
from astroquery.utils.tap.core import TapPlus
from astropy.coordinates import SkyCoord, Galactocentric
import astropy.units as u
import matplotlib
import matplotlib.patches as mpatches
import plotly.graph_objects as go
import os
plt.style.use("style.mpl")
Gaia.login(user='jadams02', password='LkCa15bcde!')

INFO: Login to gaia TAP server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]


In [5]:
K2_FILE = "k2pandc_2026.02.07_14.22.26.csv"
ANDRAE_FILE = "table_1_catwise.csv.gz"
OUTPUT_GAIA = "k2_gaia_dr3_results.csv"
OUTPUT_MERGED = "k2_gaia_andrae_merged.csv"

In [8]:
k2 = pd.read_csv(K2_FILE, comment='#')
# Extract numeric Gaia DR3 source IDs
gaia_ids = k2['gaia_dr3_id'].dropna().unique()
numeric_ids = []
for g in gaia_ids:
    m = re.search(r'DR3\s+(\d+)', str(g))
    if m:
        numeric_ids.append(int(m.group(1)))

print(f"  Unique Gaia DR3 source IDs: {len(numeric_ids)}")

  Unique Gaia DR3 source IDs: 1510


#  Query Gaia DR3 via TAP

In [9]:
upload_table = Table({'source_id': numeric_ids})

query = """
SELECT g.source_id, g.ra, g.dec, g.parallax, g.parallax_error,
    g.pmra, g.pmra_error, g.pmdec, g.pmdec_error,
    g.radial_velocity, g.radial_velocity_error,
    g.phot_g_mean_mag, g.phot_bp_mean_mag, g.phot_rp_mean_mag,
    g.ruwe, g.teff_gspphot, g.logg_gspphot, g.mh_gspphot
FROM gaiadr3.gaia_source AS g
INNER JOIN tap_upload.k2_sources AS k
ON g.source_id = k.source_id
"""

print("  Submitting async job to Gaia archive...")
job = Gaia.launch_job_async(
    query=query,
    upload_resource=upload_table,
    upload_table_name="k2_sources"
)

result = job.get_results()
gaia_df = result.to_pandas()

  Submitting async job to Gaia archive...
INFO: Query finished. [astroquery.utils.tap.core]


In [10]:
print(f"  Gaia results: {len(gaia_df)} rows")
print(f"  Stars with radial_velocity: {gaia_df['radial_velocity'].notna().sum()} / {len(gaia_df)}")
print(f"  Stars with parallax: {gaia_df['parallax'].notna().sum()} / {len(gaia_df)}")
print(f"  Stars with proper motions: {gaia_df['pmra'].notna().sum()} / {len(gaia_df)}")


  Gaia results: 1510 rows
  Stars with radial_velocity: 1232 / 1510
  Stars with parallax: 1502 / 1510
  Stars with proper motions: 1502 / 1510


In [11]:
gaia_df.to_csv(OUTPUT_GAIA, index=False)

# Crossmatch with Andrae et al. 2023 XP metallicities

In [13]:
if os.path.exists(ANDRAE_FILE):
    print(f"  Loading {ANDRAE_FILE} (this may take a moment for the full table)...")
    andrae = pd.read_csv(ANDRAE_FILE, compression='gzip' if ANDRAE_FILE.endswith('.gz') else None)
    print(f"  Andrae table: {len(andrae)} rows, {len(andrae.columns)} columns")
    print(f"  Andrae columns: {list(andrae.columns[:20])}...")

    # Identify the source_id column in Andrae (might be 'source_id', 'Source', etc.)
    andrae_id_col = None
    for col in andrae.columns:
        if 'source' in col.lower() and 'id' in col.lower():
            andrae_id_col = col
            break
    if andrae_id_col is None:
        for col in andrae.columns:
            if 'source' in col.lower():
                andrae_id_col = col
                break
    if andrae_id_col is None:
        andrae_id_col = andrae.columns[0]  # first column is often source_id

    print(f"  Using Andrae ID column: '{andrae_id_col}'")

    # Filter Andrae to only our source IDs first (much faster merge)
    source_id_set = set(gaia_df['source_id'].astype(int))
    andrae['_source_id_int'] = pd.to_numeric(andrae[andrae_id_col], errors='coerce').astype('Int64')
    andrae_matched = andrae[andrae['_source_id_int'].isin(source_id_set)].copy()
    print(f"  Andrae matches for K2 hosts: {len(andrae_matched)}")

    # Merge Gaia + Andrae
    merged = gaia_df.merge(
        andrae_matched.drop(columns=['_source_id_int']),
        left_on='source_id',
        right_on=andrae_id_col,
        how='left',
        suffixes=('_gaia', '_andrae')
    )

    # Identify the XP metallicity column
    mh_cols = [c for c in andrae_matched.columns if 'mh' in c.lower() or 'feh' in c.lower() or 'metal' in c.lower()]
    print(f"  Metallicity-related columns in Andrae: {mh_cols}")
else:
    print(f"  WARNING: {ANDRAE_FILE} not found! Skipping Andrae crossmatch.")
    print(f"  The merged file will contain only Gaia data.")
    merged = gaia_df.copy()

  Loading table_1_catwise.csv.gz (this may take a moment for the full table)...
  Andrae table: 174922161 rows, 7 columns
  Andrae columns: ['source_id', 'catwise_w1', 'catwise_w2', 'in_training_sample', 'mh_xgboost', 'teff_xgboost', 'logg_xgboost']...
  Using Andrae ID column: 'source_id'
  Andrae matches for K2 hosts: 1224
  Metallicity-related columns in Andrae: ['mh_xgboost']


## Merge altogether with planet information now

In [14]:
# Add source_id to K2 for merging
k2['source_id'] = k2['gaia_dr3_id'].apply(
    lambda x: int(re.search(r'DR3\s+(\d+)', str(x)).group(1)) if pd.notna(x) and re.search(r'DR3\s+(\d+)', str(x)) else None
)
k2['source_id'] = pd.to_numeric(k2['source_id'], errors='coerce').astype('Int64')

# Take default parameter set where available
k2_default = k2[k2['default_flag'] == 1].copy()
print(f"  K2 default parameter rows: {len(k2_default)}")

final = k2_default.merge(
    merged,
    on='source_id',
    how='left',
    suffixes=('_k2', '_gaia')
)

final.to_csv(OUTPUT_MERGED, index=False)


  K2 default parameter rows: 1806


## Summary

In [16]:
rv_mask = final['radial_velocity'].notna()
plx_mask = final['parallax'].notna() & (final['parallax'] > 0)
pm_mask = final['pmra'].notna() & final['pmdec'].notna()
toomre_ready = rv_mask & plx_mask & pm_mask
print(f"  Total K2 planets (default params): {len(final)}")
print(f"  With radial velocity:              {rv_mask.sum()}")
print(f"  With positive parallax:            {plx_mask.sum()}")
print(f"  With proper motions:               {pm_mask.sum()}")
print(f"  Ready for Toomre diagram:          {toomre_ready.sum()}")
if os.path.exists(ANDRAE_FILE):
    andrae_cols = [c for c in final.columns if 'andrae' in c.lower() or 'mh' in c.lower() or 'feh' in c.lower()]
    has_xp = final[andrae_cols].notna().any(axis=1).sum() if andrae_cols else 0
    print(f"  With Andrae XP metallicity:        {has_xp}")
print(f"\nOutput files:")
print(f"  {OUTPUT_GAIA} - Gaia DR3 astrometry only")
print(f"  {OUTPUT_MERGED} - Full merged catalog (K2 + Gaia + Andrae)")

  Total K2 planets (default params): 1806
  With radial velocity:              709
  With positive parallax:            874
  With proper motions:               875
  Ready for Toomre diagram:          705
  With Andrae XP metallicity:        861

Output files:
  k2_gaia_dr3_results.csv - Gaia DR3 astrometry only
  k2_gaia_andrae_merged.csv - Full merged catalog (K2 + Gaia + Andrae)
